In [ ]:
from pyspark.sql import SparkSession

# 1. Criar a SparkSession com suporte ao Hive
# As configurações já estão no ambiente, então o código fica limpo!
spark = (
    SparkSession.builder
    .appName("HiveMetastore-Test")
    .master("spark://spark-master:7077")
    .enableHiveSupport()  # Habilita a integração com o Hive Metastore
    .getOrCreate()
)

print("Spark Session com Hive Metastore criada!")

# 2. Criar um DataFrame de exemplo
data = [("Fusca", 1970), ("Opala", 1980), ("Chevette", 1985)]
columns = ["modelo", "ano"]
df = spark.createDataFrame(data, columns)

# 3. Salvar o DataFrame como uma tabela gerenciada
# O Spark irá gerenciar os metadados (no PostgreSQL) e os dados (no MinIO)
table_name = "carros_classicos"
print(f"Salvando DataFrame como tabela gerenciada: '{table_name}'")

df.write.mode("overwrite").saveAsTable(table_name)

print("Tabela salva com sucesso!")

# 4. Ler a tabela de volta usando spark.read.table()
print(f"\nLendo a tabela '{table_name}' do catálogo:")
df_lido = spark.read.table(table_name)
df_lido.show()

# 5. Você pode listar as tabelas existentes
print("\nTabelas no catálogo:")
spark.catalog.listTables().show()

# Você pode fechar este notebook, reiniciar o kernel ou até mesmo os contêineres.
# Ao abrir um novo notebook e executar a célula 4 novamente, a tabela ainda estará lá!

spark.stop()


In [ ]:
from pyspark.sql import SparkSession

# 1. Inicie a SparkSession com suporte ao Hive
# Lembre-se que todas as configurações já estão no ambiente, então o código fica limpo.
spark = (
    SparkSession.builder
    .appName("Gerenciando-Databases")
    .master("spark://spark-master:7077")
    .enableHiveSupport()  # Essencial para interagir com o catálogo do Hive
    .getOrCreate()
)

print("Spark Session com Hive Metastore criada!")

# 2. Crie um novo database chamado 'bronze_layer'
# Usar 'IF NOT EXISTS' é uma boa prática para evitar erros se o database já existir.
db_name = "bronze_layer"
spark.sql(f"CREATE DATABASE IF NOT EXISTS {db_name} COMMENT 'Database para dados brutos'")

print(f"\nDatabase '{db_name}' criado com sucesso!")

# 3. Verifique se o database foi criado listando todos os databases
print("\nDatabases existentes no catálogo:")
spark.sql("SHOW DATABASES").show()

# 4. Mude o contexto para o novo database
spark.sql(f"USE {db_name}")
print(f"\nContexto alterado para o database '{db_name}'.")

# 5. Crie uma tabela dentro do novo database
# Os dados desta tabela serão armazenados em MinIO no caminho:
# s3a://database/warehouse/bronze_layer.db/clientes_brutos
data = [("1", "John Doe", "2024-10-26"), ("2", "Jane Smith", "2024-10-27")]
columns = ["id_cliente", "nome_completo", "data_cadastro"]
df_clientes = spark.createDataFrame(data, columns)

df_clientes.write.mode("overwrite").saveAsTable("clientes_brutos")

print("\nTabela 'clientes_brutos' criada dentro do database 'bronze_layer'.")

# 6. Liste as tabelas no database atual para confirmar
print(f"\nTabelas no database '{db_name}':")
spark.sql("SHOW TABLES").show()

# Para acessar a tabela, você pode usar o nome qualificado: bronze_layer.clientes_brutos
# ou, como já estamos usando o database, apenas o nome da tabela.
print("\nLendo dados da nova tabela:")
spark.read.table("clientes_brutos").show()

# Lembre-se de parar a sessão ao final
spark.stop()


In [1]:
from pyspark.sql import SparkSession

# 1. Inicie a SparkSession com suporte ao Hive
# Lembre-se que todas as configurações já estão no ambiente, então o código fica limpo.
spark = (
    SparkSession.builder
    .appName("Gerenciando-Databases2")
    .master("spark://spark-master:7077")
    .enableHiveSupport()  # Essencial para interagir com o catálogo do Hive
    .getOrCreate()
)

print("Spark Session com Hive Metastore criada!")


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/24 02:03:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Spark Session com Hive Metastore criada!


In [2]:

# 2. Crie um novo database chamado 'bronze_layer'
# Usar 'IF NOT EXISTS' é uma boa prática para evitar erros se o database já existir.
db_name = "bronze_layer"
spark.sql(f"CREATE DATABASE IF NOT EXISTS {db_name} COMMENT 'Database para dados brutos'")

print(f"\nDatabase '{db_name}' criado com sucesso!")


# 4. Mude o contexto para o novo database
spark.sql(f"USE {db_name}")
print(f"\nContexto alterado para o database '{db_name}'.")

# 5. Crie uma tabela dentro do novo database
# Os dados desta tabela serão armazenados em MinIO no caminho:
# s3a://database/warehouse/bronze_layer.db/clientes_brutos
input_path = f"s3a://raw/cad_fi_hist_admin.csv"
df_clientes = spark.read.csv(input_path, sep=";", header=True)
df_clientes.show()
# df_clientes.write.mode("overwrite").saveAsTable("clientes_brutos")

# print("\nTabela 'clientes_brutos' criada dentro do database 'bronze_layer'.")

# # 6. Liste as tabelas no database atual para confirmar
# print(f"\nTabelas no database '{db_name}':")
# spark.sql("SHOW TABLES").show()

# # Para acessar a tabela, você pode usar o nome qualificado: bronze_layer.clientes_brutos
# # ou, como já estamos usando o database, apenas o nome da tabela.
# print("\nLendo dados da nova tabela:")
# spark.read.table("clientes_brutos").show()

# # Lembre-se de parar a sessão ao final
# spark.stop()

25/10/24 02:03:45 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
25/10/24 02:03:55 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
25/10/24 02:03:55 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
25/10/24 02:04:00 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
25/10/24 02:04:00 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore UNKNOWN@172.18.0.3



Database 'bronze_layer' criado com sucesso!

Contexto alterado para o database 'bronze_layer'.


25/10/24 02:04:01 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException
[Stage 1:>                                                          (0 + 1) / 1]

+------------------+----------+------------------+--------------------+------------+------------+
|        CNPJ_FUNDO|    DT_REG|        CNPJ_ADMIN|               ADMIN|DT_INI_ADMIN|DT_FIM_ADMIN|
+------------------+----------+------------------+--------------------+------------+------------+
|00.000.432/0001-00|2005-03-31|33.868.597/0001-40|    CITIBANK DTVM SA|  2005-03-31|  2005-04-01|
|00.000.432/0001-00|2005-03-31|61.472.676/0001-72|BANCO SANTANDER B...|  2005-04-01|  2006-11-21|
|00.000.432/0001-00|2005-03-31|90.400.888/0001-42|BANCO SANTANDER (...|  2006-11-21|  2008-07-18|
|00.000.744/0001-06|2005-01-13|33.066.408/0001-15|BANCO ABN AMRO RE...|  2005-01-01|  2006-06-30|
|00.000.746/0001-03|2004-12-09|60.770.336/0001-65|BANCO ALFA DE INV...|  2004-11-29|        null|
|00.000.756/0001-30|2005-03-21|17.192.451/0001-70| BANCO ITAUCARD S.A.|  2005-03-10|        null|
|00.000.777/0001-56|2005-04-14|61.510.574/0001-02|SANTANDER S.A. CO...|  2005-03-31|  2006-12-13|
|00.000.777/0001-56|

In [3]:
print(df_clientes.count())

81192
